<a href="https://colab.research.google.com/github/dharalakshmi/NNDL/blob/main/lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import zipfile
import os

# Upload the dataset
uploaded = files.upload()

# Unzip the uploaded file
zip_path = next(iter(uploaded))  # Gets uploaded file name
extract_dir = "neonatal_jaundice_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List directories
os.listdir(extract_dir)


Saving Neonatal Jaundice Assessment A Dataset of Forehead and Sternum Images for Bilirubin Estimation (1).zip to Neonatal Jaundice Assessment A Dataset of Forehead and Sternum Images for Bilirubin Estimation (1).zip


['Neonatal Jaundice Assessment A Dataset of Forehead and Sternum Images for Bilirubin Estimation']

In [39]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2

# Build a reusable model builder
def build_model():
    inp = Input(shape=(128, 128, 3))
    x = Conv2D(32, (3, 3), activation='relu')(inp)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    out = Dense(1)(x)

    model = Model(inputs=inp, outputs=out)
    return model

# Define optimizer list
opt_list = {
    "Adam": Adam(),
    "SGD": SGD(learning_rate=0.01, momentum=0.9),
    "RMSprop": RMSprop()
}

print("📊 Starting training with different optimizers...\n")

optimizer_results = {}

for name, optimizer in opt_list.items():
    print(f"🔧 Using Optimizer: {name}")

    model = build_model()  # Rebuild fresh model each time
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

    history = model.fit(X_train, y_train, epochs=5, verbose=0, validation_split=0.1)

    final_loss = history.history['val_loss'][-1]
    final_mae = history.history['val_mae'][-1]

    print(f"📉 Final Val Loss: {final_loss:.4f} | Final Val MAE: {final_mae:.4f}\n")

    optimizer_results[name] = {
        'val_loss': final_loss,
        'val_mae': final_mae
    }

print("✅ Training complete for all optimizers.")

📊 Starting training with different optimizers...

🔧 Using Optimizer: Adam
📉 Final Val Loss: 66.6058 | Final Val MAE: 7.5277

🔧 Using Optimizer: SGD
📉 Final Val Loss: nan | Final Val MAE: nan

🔧 Using Optimizer: RMSprop
📉 Final Val Loss: 29.1756 | Final Val MAE: 4.5609

✅ Training complete for all optimizers.


In [40]:
from tensorflow.keras.layers import SimpleRNN

# Example of basic RNN model
input_rnn = Input(shape=(10, 128))  # 10 time steps, 128 features
x = SimpleRNN(64, activation='tanh')(input_rnn)
x = Dense(32, activation='relu')(x)
output = Dense(1)(x)

rnn_model = Model(inputs=input_rnn, outputs=output)
rnn_model.compile(optimizer='adam', loss='mse')
rnn_model.summary()


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,465 (56.50 KB)

 Trainable params: 14,465 (56.50 KB)

 Non-trainable params: 0 (0.00 B)